In [1]:
import pandas as pd
bat2019 = pd.read_csv('Batting2019.csv', sep = ',', error_bad_lines=False)
pitch2019 = pd.read_csv('Pitching2019.csv', sep = ',', error_bad_lines=False)
field2019 = pd.read_csv('Fielding2019.csv', sep = ',', error_bad_lines=False)
field2019 = field2019.drop(['Team'], axis = 1)


In [45]:
# bat2019['Team'].unique()

array(['Dodgers', 'Angels', 'Brewers', 'Nationals', 'Rays', 'Royals',
       'White Sox', 'Athletics', 'Cardinals', 'Mets', 'Twins', 'Cubs',
       'Astros', 'Phillies', 'Braves', 'Orioles', 'Blue Jays', 'Rangers',
       'Mariners', 'Indians', 'Padres', 'Yankees', 'Red Sox',
       'Diamondbacks', 'Reds', 'Pirates', 'Tigers', 'Rockies', 'Marlins',
       'Giants', '- - -'], dtype=object)

In [2]:
# print(bat2019.columns)
# print(pitch2019.columns)
# print(field2019.columns)
batandField = pd.merge(bat2019, field2019, on =['Name'], how = 'outer' )
batFieldandPitch = pd.merge(batandField, pitch2019, on =['Team', 'Name'], how='outer')
batFieldandPitch = batFieldandPitch.fillna(0)

offMin = batFieldandPitch['Off'].min()
offMax = batFieldandPitch['Off'].max()
defxMin = batFieldandPitch['Def_x'].min()
defxMax = batFieldandPitch['Def_x'].max()
defyMin = batFieldandPitch['Def_y'].min()
defyMax = batFieldandPitch['Def_y'].max()

batFieldandPitch['normalize_Off'] = ((batFieldandPitch['Off']-offMin) / (offMax-offMin)) * (10) 
batFieldandPitch['normalize_Def_y'] = ((batFieldandPitch['Def_y']-defyMin) / (defyMax-defyMin)) * (10) 
batFieldandPitch['normalize_Def_y'] = batFieldandPitch['normalize_Def_y'] * -1
batFieldandPitch['normalize_Def_x'] = ((batFieldandPitch['Def_x']-defxMin) / (defxMax-defxMin)) * (10) 
batFieldandPitch['normalize_Def_x'] = batFieldandPitch['normalize_Def_x'] * -1

# NewDef = (batFieldandPitch[['normalize_Def_x', 'Team']].groupby('Team', as_index=False).mean()['normalize_Def_x'] + batFieldandPitch[['normalize_Def_y', 'Team']].groupby('Team', as_index=False).mean()['normalize_Def_y'])/2
# print( batFieldandPitch[['normalize_Off', 'Team']].groupby('Team', as_index=False).sum()['normalize_Off'])
# print( batFieldandPitch[['normalize_Def_y', 'Team']].groupby('Team', as_index=False).sum()['normalize_Def_y'])
# print(batFieldandPitch[['normalize_Def_x', 'Team']].groupby('Team', as_index=False).sum()['normalize_Def_x']) 

# batFieldandPitch.head([batFieldandPitch[['normalize_Off', 'Team']].groupby('Team', as_index=False).sum()['Team'], batFieldandPitch[['normalize_Off', 'Team']].groupby('Team', as_index=False).sum()['normalize_Off'] + batFieldandPitch[['normalize_Def_y', 'Team']].groupby('Team', as_index=False).sum()['normalize_Def_y'] + batFieldandPitch[['normalize_Def_x', 'Team']].groupby('Team', as_index=False).sum()['normalize_Def_x']])

newStat = pd.concat([batFieldandPitch[['normalize_Off', 'Team']].groupby('Team', as_index=False).mean()['Team'], batFieldandPitch[['normalize_Off', 'Team']].groupby('Team', as_index=False).mean()['normalize_Off'] + (batFieldandPitch[['normalize_Def_y', 'Team']].groupby('Team', as_index=False).mean()['normalize_Def_y'] + batFieldandPitch[['normalize_Def_x', 'Team']].groupby('Team', as_index=False).mean()['normalize_Def_x'])/2], axis=1)

newStat = newStat.drop(newStat.index[[0,1]])
newStat['teamID'] =  ['LAA', 'HOU', 'OAK', 'TOR', 'ATL', 'MIL', 'STL', 'CHC', 'ARI', 'LAD', 'SFG', 'CLE', 'SEA', 'MIA', 'NYM', 'WSN', 'BAL', 'SDP', 'PHI', 'PIT', 'TEX', 'TBR', 'BOS', 'CIN', 'COL', 'KCR', 'DET', 'MIN', 'CHW', 'NYY']
print(newStat)
# print(batFieldandPitch.columns)

            Team         0 teamID
2         Angels -2.475038    LAA
3         Astros -1.644265    HOU
4      Athletics -2.622888    OAK
5      Blue Jays -2.465678    TOR
6         Braves -1.576181    ATL
7        Brewers -2.259636    MIL
8      Cardinals -2.021505    STL
9           Cubs -1.887865    CHC
10  Diamondbacks -2.616274    ARI
11       Dodgers -1.406490    LAD
12        Giants -3.990495    SFG
13       Indians -2.159041    CLE
14      Mariners  0.284702    SEA
15       Marlins -3.416112    MIA
16          Mets -1.624040    NYM
17     Nationals -1.356567    WSN
18       Orioles -2.118856    BAL
19        Padres -3.382708    SDP
20      Phillies -2.475562    PHI
21       Pirates -2.723374    PIT
22       Rangers -2.230116    TEX
23          Rays -1.840001    TBR
24       Red Sox -1.980470    BOS
25          Reds -2.872230    CIN
26       Rockies -3.887529    COL
27        Royals -2.104804    KCR
28        Tigers -3.011625    DET
29         Twins -1.935697    MIN
30     White S

In [3]:
#Download the Lahman data
from pybaseball.lahman import *
teams = teams()
teams2016 = teams.loc[teams['yearID']==2016]
teams2016 = teams2016[['teamID', 'W', 'attendance']]
newStat = pd.merge(newStat, teams2016, on='teamID')
salaries = salaries()
salaries = salaries.loc[salaries['yearID'] == 2016]
salaries = salaries.groupby('teamID', as_index=False).sum()
salaries = salaries[['teamID', 'salary']]
newStat = pd.merge(newStat, salaries, on='teamID')
print(newStat)

            Team         0 teamID   W  attendance     salary
0         Angels -2.475038    LAA  74   3016142.0  137251333
1         Astros -1.644265    HOU  84   2306623.0   94893700
2      Athletics -2.622888    OAK  69   1521506.0   86806234
3      Blue Jays -2.465678    TOR  89   3392099.0  138701700
4         Braves -1.576181    ATL  68   2020914.0   68498291
5        Brewers -2.259636    MIL  73   2314614.0   68775237
6   Diamondbacks -2.616274    ARI  69   2036216.0   87439063
7        Indians -2.159041    CLE  94   1591667.0   74311900
8       Mariners  0.284702    SEA  86   2267928.0  135683339
9        Marlins -3.416112    MIA  79   1712417.0   77314202
10       Orioles -2.118856    BAL  89   2172344.0  161863456
11      Phillies -2.475562    PHI  71   1915144.0   58980000
12       Pirates -2.723374    PIT  78   2249201.0  103778833
13       Rangers -2.230116    TEX  95   2710402.0  176038723
14       Red Sox -1.980470    BOS  93   2955434.0  188545761
15          Reds -2.8722

,Name,Team,G,PA,HR,R,RBI,SB,BB%,K%,...,AVG,OBP,SLG,wOBA,wRC+,BsR,Off,Def,WAR,playerid
0,Cody Bellinger,Dodgers,18,78,9,22,23,3,12.8 %,11.5 %,...,0.433,0.513,0.925,0.577,264,-0.1,16.7,-0.4,1.9,15998
1,Mike Trout,Angels,13,54,5,10,12,1,29.6 %,7.4 %,...,0.382,0.593,0.882,0.563,272,0.5,11.8,2.4,1.6,10155
2,Christian Yelich,Brewers,17,78,8,17,22,3,14.1 %,14.1 %,...,0.354,0.449,0.785,0.490,202,1.0,11.4,-0.1,1.4,11477
3,Anthony Rendon,Nationals,14,63,6,18,17,0,9.5 %,12.7 %,...,0.400,0.460,0.873,0.534,230,0.9,11.7,-1.5,1.2,12861
4,Austin Meadows,Rays,15,64,6,8,17,2,12.5 %,21.9 %,...,0.357,0.438,0.732,0.474,211,0.5,9.1,-0.4,1.1,15672


In [40]:
# pitch2019.head()

,Name,Team,W,L,SV,G,GS,IP,K/9,BB/9,HR/9,BABIP,LOB%,GB%,HR/FB,ERA,FIP,xFIP,WAR,playerid
0,Matthew Boyd,Tigers,1,1,0,3,3,17.1,15.06,3.12,0.00,0.389,71.4 %,26.5 %,0.0 %,2.60,1.01,2.62,1.1,15440
1,Max Scherzer,Nationals,1,2,0,4,4,27.0,11.67,1.67,0.67,0.344,70.9 %,48.5 %,9.5 %,3.33,2.29,2.78,1.0,3137
2,Carlos Rodon,White Sox,2,2,0,4,4,22.0,11.86,3.68,0.00,0.281,61.5 %,41.1 %,0.0 %,3.27,1.87,3.74,1.0,16137
3,Luis Castillo,Reds,1,1,0,4,4,24.2,11.68,4.74,0.00,0.184,83.3 %,58.7 %,0.0 %,1.46,2.37,3.36,0.9,15689
4,Jose Berrios,Twins,2,1,0,4,4,27.1,9.22,1.65,0.99,0.185,83.3 %,43.3 %,11.1 %,2.30,3.18,3.60,0.8,14168


In [41]:
# field2019.head()

,Name,Team,Pos,Inn,rSZ,rCERA,rSB,rGDP,rARM,rGFP,...,FSR,FRM,ARM,DPR,RngR,ErrR,UZR,UZR/150,Def,playerid
0,Buster Posey,SFG,C,99.0,1,0.0,1.0,NaN,NaN,1,...,NaN,1.7,NaN,NaN,NaN,NaN,NaN,NaN,3.5,9166
1,Ramon Laureano,OAK,CF,153.2,0,NaN,NaN,NaN,4.0,1,...,NaN,NaN,3.9,NaN,-0.5,-0.2,3.3,47.3,3.5,17128
2,J.T. Realmuto,PHI,C,117.0,0,0.0,1.0,NaN,NaN,1,...,NaN,1.1,NaN,NaN,NaN,NaN,NaN,NaN,3.1,11739
3,Martin Maldonado,KCR,C,115.0,1,0.0,1.0,NaN,NaN,2,...,NaN,1.1,NaN,NaN,NaN,NaN,NaN,NaN,3.0,6887
4,Harrison Bader,STL,CF,117.0,0,NaN,NaN,NaN,1.0,-1,...,NaN,NaN,1.7,NaN,1.6,-0.6,2.7,44.9,2.9,18030


In [5]:
# print(batFieldandPitch[['Off','Def_x', 'Def_y']])

KeyError: "['off'] not in index"